In [7]:
from google.colab import files

In [10]:
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nwadiaromiracle","key":"e28b1d8c88dcea9c233fb4bfb819cd9d"}'}

In [11]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [12]:
!kaggle competitions list | grep dogs

In [13]:
!kaggle competitions download -c dogs-vs-cats

401 Client Error: Unauthorized for url: https://www.kaggle.com/api/v1/competitions/data/download-all/dogs-vs-cats


In [14]:
import os
import zipfile
import requests

In [15]:
url = "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [16]:
zip_path = "cats_and_dogs.zip"

In [18]:
!wget --no-check-certificate {url} -O {zip_path}

--2026-01-20 14:04:37--  https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip
Resolving download.microsoft.com (download.microsoft.com)... 23.209.58.12, 2600:1408:ec00:89b::317f, 2600:1408:ec00:887::317f
Connecting to download.microsoft.com (download.microsoft.com)|23.209.58.12|:443... connected.
  Unable to locally verify the issuer's authority.
HTTP request sent, awaiting response... 200 OK
Length: 824887076 (787M) [application/octet-stream]
Saving to: ‘cats_and_dogs.zip’

cats_and_dogs.zip   100%[===================>] 786.67M   229MB/s    in 3.4s    

2026-01-20 14:04:40 (232 MB/s) - ‘cats_and_dogs.zip’ saved [824887076/824887076]



In [19]:
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall("data")

In [20]:
os.rename("data/PetImages", "data/train")

In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np
from PIL import Image

# Data transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

def is_valid_file(path):
    try:
        with Image.open(path) as img:
            img.verify()
        return True
    except:
        return False

# Load dataset (assumes images in ./data/train/cat.* and ./data/train/dog.*)
dataset = datasets.ImageFolder(root='./data/train', transform=transform, is_valid_file=is_valid_file)
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [26]:
class CustomCNN(nn.Module):
    def __init__(self):
        super(CustomCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  # After pooling: 128x128 -> 64->32->16
        self.fc2 = nn.Linear(512, 2)  # Binary output
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.relu(self.conv3(x))
        x = self.pool(x)
        x = x.view(-1, 128 * 16 * 16)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.fc2(x)
        return x

cnn_model = CustomCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

In [ ]:
def train_model(model, loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f'Epoch {epoch+1}, Loss: {running_loss / len(loader):.4f}')

train_model(cnn_model, train_loader)

# Evaluate
def evaluate_model(model, loader):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for inputs, labels in loader:
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.numpy())
    acc = accuracy_score(true, preds)
    print(f'Accuracy: {acc:.4f}')
    return preds, true

cnn_preds, cnn_true = evaluate_model(cnn_model, val_loader)

In [ ]:
def visualize_predictions(loader, model, num_images=5):
    model.eval()
    images, labels = next(iter(loader))
    images, labels = images[:num_images].to(device), labels[:num_images]
    outputs = model(images)
    _, preds = torch.max(outputs, 1)
    images = images.cpu().numpy().transpose(0, 2, 3, 1)  # Denormalize for display
    for i in range(num_images):
        plt.imshow(images[i])
        plt.title(f'Pred: {"Dog" if preds[i] == 1 else "Cat"}, True: {"Dog" if labels[i] == 1 else "Cat"}')
        plt.show()

visualize_predictions(val_loader, cnn_model)

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(3 * 128 * 128, 512)  # Flatten input
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 2)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = x.view(-1, 3 * 128 * 128)
        x = self.dropout(self.relu(self.fc1(x)))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

mlp_model = MLP().to(device)
mlp_optimizer = optim.Adam(mlp_model.parameters(), lr=0.001)

train_model(mlp_model, train_loader)  # Reuse train function
mlp_preds, mlp_true = evaluate_model(mlp_model, val_loader)

# Compare accuracies
print("CNN vs MLP Accuracy:", accuracy_score(cnn_true, cnn_preds), "vs", accuracy_score(mlp_true, mlp_preds))